In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator, 
    DirectoryIterator
)
from google.cloud import aiplatform, storage
from keras.applications.resnet50 import ResNet50, preprocess_input

import pathlib

import sys
from loguru import logger

sys.path.insert(0, "./backend")
import recognition
import json

In [2]:
PROJECT_ID="celebrity-face-recognition"

In [3]:
path_to_saved_model = pathlib.Path("./backend/model")

train=False
if not train:
    assert path_to_saved_model.exists(), f"Could not find {path_to_saved_model}"
    assert path_to_saved_model.is_dir(), f"{path_to_saved_model} is not a directory"
    model = recognition.load_local_model(path_to_saved_model)

First, download the pre-trained ResNet 50 model from TensorFlow Hub

In [7]:
base_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(*recognition.DEFAULT_IMAGE_SIZE, 3),
)

Then, figure out how many celebrities we have (just the number of directories inside the `Celebrity Faces Dataset`)

In [8]:
path_to_celebrity_dataset = pathlib.Path("./backend/Celebrity Faces Dataset/")
assert path_to_celebrity_dataset.exists(), f"Could not find {path_to_celebrity_dataset}."
number_celebrities = len(list(path_to_celebrity_dataset.iterdir()))
print(f"There are {number_celebrities} celebrities in the dataset.")

There are 17 celebrities in the dataset.


Now, we'll instantiate a model on top of our pre-trained ResNet 50 model.

In [9]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(number_celebrities, activation="softmax"),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 17)                34833     
                                                                 
Total params: 23622545 (90.11 MB)
Trainable params: 23569425 (89.91 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


Now we'll use TensorFlow utilities (`ImageDataGenerator`) to load and preprocess the celebrity images

In [10]:
target_image_size = recognition.DEFAULT_IMAGE_SIZE

training_data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

training_data_streamer: DirectoryIterator = training_data_generator.flow_from_directory(
    path_to_celebrity_dataset,
    target_size=target_image_size,
    batch_size=32,
    class_mode="categorical",
)


Found 1800 images belonging to 17 classes.


Now we'll compile our model, using the Adam optimizer, and the categorical crossentropy loss function

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [12]:
training_epochs = 10

history = model.fit(
    training_data_streamer,
    epochs=training_epochs,
)

Epoch 1/10
57/57 [==============================] - 114s 2s/step - loss: 2.2561 - accuracy: 0.3028
Epoch 2/10
57/57 [==============================] - 110s 2s/step - loss: 0.9090 - accuracy: 0.7183
Epoch 3/10
57/57 [==============================] - 111s 2s/step - loss: 0.4521 - accuracy: 0.8767
Epoch 4/10
57/57 [==============================] - 89s 2s/step - loss: 0.2322 - accuracy: 0.9439
Epoch 5/10
57/57 [==============================] - 109s 2s/step - loss: 0.1618 - accuracy: 0.9633
Epoch 6/10
57/57 [==============================] - 137s 2s/step - loss: 0.1127 - accuracy: 0.9717
Epoch 7/10
57/57 [==============================] - 168s 3s/step - loss: 0.0852 - accuracy: 0.9806
Epoch 8/10
57/57 [==============================] - 128s 2s/step - loss: 0.0757 - accuracy: 0.9822
Epoch 9/10
57/57 [==============================] - 122s 2s/step - loss: 0.0575 - accuracy: 0.9850
Epoch 10/10
57/57 [==============================] - 143s 3s/step - loss: 0.0514 - accuracy: 0.9856


Now, we use this fine-tuned model to make predictions:

In [13]:
example_image = recognition.load_image_from_file(
    image_path=path_to_celebrity_dataset / "Will Smith/001_beebcee2.jpg"
)

idx_to_celebrity_name = {str(v): k for k, v in training_data_streamer.class_indices.items()}

predicted_celebrity = recognition.make_prediction(example_image, model, idx_to_celebrity_name)
print(f"Predicted celebrity: {predicted_celebrity}")

2023-12-29 12:14:40.716 | INFO     | recognition:make_prediction:79 - Input image: <PIL.Image.Image image mode=RGB size=112x112 at 0x2C8E753D0>
2023-12-29 12:14:40.738 | INFO     | recognition:make_prediction:81 - Image array shape: (112, 112, 3)
2023-12-29 12:14:40.765 | INFO     | recognition:make_prediction:83 - After expanding dimensions: (1, 112, 112, 3)
2023-12-29 12:14:40.799 | INFO     | recognition:make_prediction:85 - After preprocessing: (1, 112, 112, 3)


1/1 [==============================] - 4s 4s/step


2023-12-29 12:14:46.241 | INFO     | recognition:make_prediction:90 - Predicted celebrity index: 16
2023-12-29 12:14:46.246 | INFO     | recognition:make_prediction:92 - Using idx_to_celebrity_name mapping: {'0': 'Angelina Jolie', '1': 'Brad Pitt', '2': 'Denzel Washington', '3': 'Hugh Jackman', '4': 'Jennifer Lawrence', '5': 'Johnny Depp', '6': 'Kate Winslet', '7': 'Leonardo DiCaprio', '8': 'Megan Fox', '9': 'Natalie Portman', '10': 'Nicole Kidman', '11': 'Robert Downey Jr', '12': 'Sandra Bullock', '13': 'Scarlett Johansson', '14': 'Tom Cruise', '15': 'Tom Hanks', '16': 'Will Smith'}
2023-12-29 12:14:46.248 | INFO     | recognition:make_prediction:98 - Predicted celebrity name: Will Smith


Predicted celebrity: Will Smith


Save our fine-tuned ResNet 50 to a `model` directory in the `backend` folder in the SavedModel format, making sure to attach the index to celebrity name dictionary as an asset inside the model.

In [14]:
path_to_idx_to_celebrity_mapping = path_to_saved_model.parent / "idx_to_celebrity_name.json"
with open(path_to_idx_to_celebrity_mapping, 'w') as f:
    json.dump(idx_to_celebrity_name, f)
asset = tf.saved_model.Asset(path_to_idx_to_celebrity_mapping)

model.asset = asset
model.save(path_to_saved_model)

INFO:tensorflow:Assets written to: backend/model/assets


INFO:tensorflow:Assets written to: backend/model/assets


And here is an example of how you'd load the model back in, along with its dictionary of labels:

In [15]:
loaded_model = tf.keras.models.load_model(path_to_saved_model)
with open(loaded_model.asset.asset_path.numpy(), 'r') as f:
    loaded_idx_to_celebrity_name= json.load(f)

print(loaded_idx_to_celebrity_name)

{'0': 'Angelina Jolie', '1': 'Brad Pitt', '2': 'Denzel Washington', '3': 'Hugh Jackman', '4': 'Jennifer Lawrence', '5': 'Johnny Depp', '6': 'Kate Winslet', '7': 'Leonardo DiCaprio', '8': 'Megan Fox', '9': 'Natalie Portman', '10': 'Nicole Kidman', '11': 'Robert Downey Jr', '12': 'Sandra Bullock', '13': 'Scarlett Johansson', '14': 'Tom Cruise', '15': 'Tom Hanks', '16': 'Will Smith'}


## Deploy to GCP

In [4]:
from google.cloud import storage

def upload_dir_to_gcs(local_path, bucket_name, gcs_path):
  """Upload local directory to GCS"""
  if isinstance(local_path, str):
    local_path = pathlib.Path(local_path)
  if isinstance(gcs_path, str):
    gcs_path = pathlib.Path(gcs_path)

  uploaded_objects = []

  client = storage.Client(project=PROJECT_ID)
  bucket = client.bucket(bucket_name)

  for local_file in local_path.glob("**/*"):
    remote_path = "/".join(local_file.parts[2:])
    if not local_file.is_dir():
      blob = bucket.blob(remote_path)
      blob.upload_from_filename(local_file)
      uploaded_objects.append(local_file)
  
  logger.info(f"Uploaded {local_path} to {gcs_path} in {bucket_name}")
  logger.info(f"Uploaded {len(uploaded_objects)} objects")

bucket_name = "celebrity-face-detection-model-bucket"
model_cloud_storage_uri = f"gs://{bucket_name}/model"

upload_dir_to_gcs(
    local_path=path_to_saved_model,
    bucket_name=bucket_name,
    gcs_path=model_cloud_storage_uri
)

NotFound: 404 POST https://storage.googleapis.com/upload/storage/v1/b/celebrity-face-detection-model-bucket/o?uploadType=multipart: {
  "error": {
    "code": 404,
    "message": "The specified bucket does not exist.",
    "errors": [
      {
        "message": "The specified bucket does not exist.",
        "domain": "global",
        "reason": "notFound"
      }
    ]
  }
}
: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>)

In [ ]:
aiplatform.init(project=PROJECT_ID, location="us-east4")
model = aiplatform.Model.upload(
    display_name="celebrity-recognition",
    artifact_uri=model_cloud_storage_uri,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest",
)
model

In [ ]:
endpoint = model.deploy(
    deployed_model_display_name="celebrity_recognition",
    machine_type="n1-standard-4",
    accelerator_type=None,
    accelerator_count=0
)